# Get the VIT_B sam model first

In [1]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

--2023-06-17 17:51:01--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 65.8.11.91, 65.8.11.109, 65.8.11.16, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|65.8.11.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  16.1MB/s    in 22s     

2023-06-17 17:51:24 (16.1 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



## Set-up

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
# Get a list of all the image files in the folder
image_folder = '/home/noahr/repos/personal_projects/img_to_svg/Images'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')]

## Automatic mask generation

To run automatic mask generation, provide a SAM model to the `SamAutomaticMaskGenerator` class. Set the path below to the SAM checkpoint. Running on CUDA and with the default model is recommended.

In [3]:
import sys
import os
import json
sys.path.append("..")
import pycocotools.mask as maskUtils
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "models/sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

ModuleNotFoundError: No module named 'idna'

To generate masks, just run `generate` on an image.

In [5]:
# Loop through each image file and process it with mask_generator
for image_file in image_files:
    image = cv2.imread(image_file)
    mask_dict_list = mask_generator.generate(image)
    
    # Loop through each dictionary in the mask_dict_list array
    for i, mask_dict in enumerate(mask_dict_list):
        # Check if the segmentation key exists in the mask_dict dictionary
        if 'segmentation' in mask_dict:
            # Save the mask numpy array to a file
            mask_folder = '/home/noahr/repos/personal_projects/img_to_svg/masks'
            mask_file = os.path.join(mask_folder, os.path.basename(image_file).replace('.jpg', f'_mask_{i}.npy'))
            np.save(mask_file, mask_dict['segmentation'])
            
            # Load the mask numpy array from the file
            mask = np.load(mask_file)
            
            # Convert the mask numpy array to a binary mask
            mask_binary = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
            mask_binary[mask > 0] = 1
            
            # Convert the binary mask to a COCO RLE format
            mask_rle = maskUtils.encode(np.asfortranarray(mask_binary))
            
            # Extract the counts key from the mask RLE dictionary
            counts = mask_rle['counts']
            
            # Create a new dictionary with the required keys for COCO RLE format
            mask_coco_rle = {
                'size': [mask.shape[0], mask.shape[1]],
                'counts': counts.decode('utf-8'),
            }
            
            # Save the mask COCO RLE to a file
            mask_rle_file = os.path.join(mask_folder, os.path.basename(image_file).replace('.jpg', f'_mask_{i}.json'))
            with open(mask_rle_file, 'w') as f:
                json.dump(mask_coco_rle, f)
            
            # Delete the mask numpy array file
            os.remove(mask_file)
        else:
            print(f"Error: segmentation key not found in mask_dict for {image_file}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.17 GiB (GPU 0; 6.00 GiB total capacity; 3.68 GiB already allocated; 0 bytes free; 5.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Create cutouts of every mask in the image, store them in cutouts folder

In [6]:
import os
import numpy as np
import pycocotools.mask as maskUtils
import json
import cv2
import matplotlib.pyplot as plt

# Define the paths to the images and masks folders
images_path = '/home/noahr/repos/segment-anything/notebooks/images'
masks_path = '/home/noahr/repos/segment-anything/notebooks/masks'
cutouts_path = '/home/noahr/repos/segment-anything/notebooks/cutouts'

# Loop through all the files in the images folder
for filename in os.listdir(images_path):
    # Load the image
    image = cv2.imread(os.path.join(images_path, filename))
    
    # Load the corresponding mask
    mask_number = 0
    mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(mask_number)
    while os.path.exists(os.path.join(masks_path, mask_filename)):
        with open(os.path.join(masks_path, mask_filename), 'r') as f:
            rle_dict = json.load(f)
        size = rle_dict['size']
        counts = rle_dict['counts']
        mask_decoded = maskUtils.decode({'size': size, 'counts': counts})
        mask_binary = np.zeros((size[0], size[1]), dtype=np.uint8)
        mask_binary[mask_decoded > 0] = 1

        # Extract the cutout from the image using the mask
        cutout = image * mask_decoded[..., np.newaxis]

        # Save the cutout to the cutouts folder
        cutout_filename = os.path.splitext(filename)[0] + '_cutout_{}.jpg'.format(mask_number)
        cv2.imwrite(os.path.join(cutouts_path, cutout_filename), cutout)
        
        # Delete the mask file
        os.remove(os.path.join(masks_path, mask_filename))
        
        # Increment the mask number
        mask_number += 1
        mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(mask_number)